## Readme
### 目的：实现定期爬取贝壳网上租房数据，进行周度对比整租房源的下架(出租)情况、楼盘出租率、楼盘普租价格等情况。
### 思路描述：
### 整体观测：贝壳网后台数据是自动从数据库里抓取的；比如我选中【深圳】-【南山区】-【竹子林】，共有250套房源，每页呈现30套房源数据，这30套在每一次点击进去数据都会略有不同。
### url方面：采用静态编号pg[ ]代表页数，zufang/后面携带商圈&行政区的拼音编码；所以首先根据所爬内容创建一个商圈list；之所以没用行政区纬度去采集是因为商圈的范围小，为了保障数据的完整性，在小商圈中采集效果更好。
### 采集内容方面： 值得注意的是我们在整租房源下进行采集，整租房源编号在url上保持不变就可以；为了保障数据采集且能够有效去重，在图片命名上能够采集到一个类似房源编号的东西：SZXXXXXXXXX；这个数据能够帮我们做唯一性校验；
### 基本方法：request + beautifulsoup

## 一、导入所需要的包

In [1]:
import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import random
import pymysql

## 二、设置一个贝壳商圈List

In [3]:
#贝壳商圈名称 手动一个个查验，因为有的商圈名称不仅仅是拼音还带数字（全国重名的商圈）
business_area = ['yinhu','bagualing','baihua','chegongmiao','chiwei','futianbaoshuiqu','futianzhongxin','huanggang','huangmugang','huaqiangbei','huaqiangnan','jingtian','lianhua','meilin','shixia','shangxiasha',
                 'shawei','shangbu','xiangmihu','xiangmeibei','xinzhou1','yuanling','yinhu','zhuzilin',
                 'baishizhou','daxuecheng3','hongshuwan','houhai','huaqiaocheng1','kejiyuan','nanshanzhongxin','nantou','qianhai','shekou','shenzhenwan','xili1',
                 'baoanzhongxin','bihai1','fanshen','fuyong','songgang','shajing','shiyan','taoyuanju','xinan','xicheng1','xixiang',
                 'bantian','guanlan','hongshan6','longhuazhongxin','longhuaxinqu','minzhi','meilinguan','shangtang','shiyan']
#Count 一共采集多少个商圈
len(business_area)

56

## 三、采集数据封包

In [3]:
#方案二（省去清洗） 
def house_info_2(single_soup):
    # 设立一个List
    house_info = []
    # 整个页面的data Elements
    total_div = single_soup.find_all('div',class_="content__list--item--main")
    for house in total_div:
        try:
            #房源编号
            house_url = house.find('a',class_='twoline').get('href').replace('/zufang/','').replace('.html','')
            #房屋名称
            house_title = house.find('a',class_="twoline").text.split(" ")[10]
            #房屋户型
            house_type = house.find('a',class_="twoline").text.split(" ")[11]
            #房屋格局（复式、跃层等等）
            house_aspect = house.find('a',class_="twoline").text.split(" ")[12]
            #房屋行政区
            house_district = house.find_all('a',target="_blank")[1].text
            #房屋商圈
            house_business_area =house.find_all('a',target="_blank")[2].text
            #楼盘名称
            resblock_name = house.find_all('a',target="_blank")[3].text
            #房屋面积
            house_area = house.find('p',class_="content__list--item--des").text.split("/")[1].replace(" ","").replace('\n', '')
            #房屋朝向（东南西北）
            house_aspect_2 = house.find('p',class_="content__list--item--des").text.split("/")[2].replace(" ","").replace('\n', '')
            #房屋户型+卫生间个数
            house_type_2 = house.find('p',class_="content__list--item--des").text.split("/")[3].replace(" ","").replace('\n', '')
            #房屋楼层
            house_floor = house.find('p',class_="content__list--item--des").text.split("/")[4].replace(" ","").replace('\n', '')
            #贝壳标签Label
            house_label = house.find('p',class_="content__list--item--bottom oneline").text.replace('\n', ' ')
            #是否品牌方-Brand
            house_brand = house.find('p',class_="content__list--item--brand oneline").text.replace('\n', ' ')
            #价格Price
            house_price = house.find('span',class_="content__list--item-price").text.replace(' 元/月','')
            # 汇总
            list1 = [house_title,house_type,house_aspect,house_district,house_business_area,resblock_name,house_area,house_aspect_2,
                    house_type_2,house_floor,house_label,house_brand,house_price,house_url]
            house_info.append(list1)
            #防止数据丢失
        except (IndexError,AttributeError,AttributeError,AttributeError,AttributeError):
            pass
    return house_info


In [4]:
#Count是用来计算循环页数
count = 0
#house_all_info是用来过渡采集数据
house_all_info = []
#在商圈内循环
for single_business_area in business_area:
    i = 1
    #bug_times 用来尽量在同商圈内使得采集数据更全，逻辑在下面
    bug_times = 1
    single_url = 'https://sz.zu.ke.com/zufang/{}/pg{}rt200600000001/#contentList'.format(single_business_area,i)
    while count<=5000:
        time.sleep(0.5)
        print('开始收集',single_url)
        single_responce = requests.get(single_url)
        single_soup = bs(single_responce.text,'lxml')
        house_all_info.append( house_info_2(single_soup))
        
        try:
            #贝壳网页上显示页数（用来判断）
            pages = int(single_soup.find('div',class_='content__pg').get("data-totalpage"))
            #贝壳网页上显示套数（用来判断）
            total_num = int(single_soup.find('span',class_='content__title--hl').text)
            #商圈中文名称（用来判断）
            chinese_sba = single_soup.find('p',class_='content__title').text.replace(' ','').split('\n')[2].replace('深圳','').replace('租房','')
        except (IndexError,AttributeError,AttributeError,AttributeError,AttributeError,KeyboardInterrupt,KeyboardInterrupt,KeyboardInterrupt):
            pass
        i += 1
        
        final_house_info = []
        for house_info in house_all_info:
            for house_single_info in house_info:
                final_house_info.append(house_single_info)
        test_df = pd.DataFrame(final_house_info).drop_duplicates()
        print(chinese_sba)
        print(len(test_df[test_df[4]==chinese_sba]))
        #先挨个页数爬取    
        if i <= pages:
            single_url = 'https://sz.zu.ke.com/zufang/{}/pg{}rt200600000001/#contentList'.format(single_business_area,i)
        #如果不够显示套数再在此商圈页面爬取最高不超过80页次
        elif len(test_df[test_df[4]==chinese_sba]) < (total_num) and bug_times <=80:
            single_url = 'https://sz.zu.ke.com/zufang/{}/pg{}rt200600000001/#contentList'.format(single_business_area,random.randint(1,pages))
            bug_times += 1
        else:
            break
        count += 1
        print('收集第{}页完毕'.format(count))

开始收集 https://sz.zu.ke.com/zufang/yinhu/pg1rt200600000001/#contentList
银湖
30
收集第1页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg2rt200600000001/#contentList
银湖
47
收集第2页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
61
收集第3页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
61
收集第4页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
61
收集第5页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg1rt200600000001/#contentList
银湖
65
收集第6页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
70
收集第7页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg2rt200600000001/#contentList
银湖
75
收集第8页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
75
收集第9页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg3rt200600000001/#contentList
银湖
75
收集第10页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg2rt200600000001/#contentList
银湖
75
收集第11页完毕
开始收集 https://sz.zu.ke.com/zufang/yinhu/pg2rt200600000001/#contentList
银湖
7

赤尾
57
收集第88页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg2rt200600000001/#contentList
赤尾
59
收集第89页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg3rt200600000001/#contentList
赤尾
59
收集第90页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg3rt200600000001/#contentList
赤尾
59
收集第91页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg2rt200600000001/#contentList
赤尾
59
收集第92页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg2rt200600000001/#contentList
赤尾
59
收集第93页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg2rt200600000001/#contentList
赤尾
59
收集第94页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg3rt200600000001/#contentList
赤尾
59
收集第95页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg2rt200600000001/#contentList
赤尾
59
收集第96页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg1rt200600000001/#contentList
赤尾
59
收集第97页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg3rt200600000001/#contentList
赤尾
59
收集第98页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei/pg1rt200600000001/#contentList
赤尾
59
收集第99页完毕
开始收集 https://sz.zu.ke.com/zufang/chiwei

开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
113
收集第180页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
113
收集第181页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg3rt200600000001/#contentList
福田保税区
113
收集第182页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
113
收集第183页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg3rt200600000001/#contentList
福田保税区
115
收集第184页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
115
收集第185页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
115
收集第186页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg4rt200600000001/#contentList
福田保税区
115
收集第187页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg1rt200600000001/#contentList
福田保税区
115
收集第188页完毕
开始收集 https://sz.zu.ke.com/zufang/futianbaoshuiqu/pg2rt200600000001/#contentList
福田保税区
117
开始收集 https

皇岗
337
收集第262页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg15rt200600000001/#contentList
皇岗
356
收集第263页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg16rt200600000001/#contentList
皇岗
370
收集第264页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg17rt200600000001/#contentList
皇岗
379
收集第265页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg18rt200600000001/#contentList
皇岗
387
收集第266页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg19rt200600000001/#contentList
皇岗
398
收集第267页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg20rt200600000001/#contentList
皇岗
420
收集第268页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg21rt200600000001/#contentList
皇岗
428
收集第269页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg17rt200600000001/#contentList
皇岗
436
收集第270页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg5rt200600000001/#contentList
皇岗
436
收集第271页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg18rt200600000001/#contentList
皇岗
450
收集第272页完毕
开始收集 https://sz.zu.ke.com/zufang/huanggang/pg19rt200600000001/#c

华强北
148
收集第349页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg3rt200600000001/#contentList
华强北
148
收集第350页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg3rt200600000001/#contentList
华强北
148
收集第351页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg1rt200600000001/#contentList
华强北
148
收集第352页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg5rt200600000001/#contentList
华强北
148
收集第353页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg5rt200600000001/#contentList
华强北
148
收集第354页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg1rt200600000001/#contentList
华强北
148
收集第355页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg4rt200600000001/#contentList
华强北
148
收集第356页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg5rt200600000001/#contentList
华强北
148
收集第357页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg4rt200600000001/#contentList
华强北
148
收集第358页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangbei/pg3rt200600000001/#contentList
华强北
148
收集第359页完毕
开始收集 https://sz.zu.ke.com/zufang/huaqiangb

...

龙华中心
716
收集第3185页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg2rt200600000001/#contentList
龙华中心
722
收集第3186页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg30rt200600000001/#contentList
龙华中心
722
收集第3187页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg21rt200600000001/#contentList
龙华中心
725
收集第3188页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg19rt200600000001/#contentList
龙华中心
729
收集第3189页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg4rt200600000001/#contentList
龙华中心
732
收集第3190页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg5rt200600000001/#contentList
龙华中心
735
收集第3191页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg27rt200600000001/#contentList
龙华中心
735
收集第3192页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg7rt200600000001/#contentList
龙华中心
738
收集第3193页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg17rt200600000001/#contentList
龙华中心
740
收集第3194页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuazhongxin/pg12rt200600000001/#conten

龙华新区
290
收集第3268页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg12rt200600000001/#contentList
龙华新区
290
收集第3269页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg11rt200600000001/#contentList
龙华新区
290
收集第3270页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg8rt200600000001/#contentList
龙华新区
290
收集第3271页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg8rt200600000001/#contentList
龙华新区
290
收集第3272页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg2rt200600000001/#contentList
龙华新区
290
收集第3273页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg10rt200600000001/#contentList
龙华新区
290
收集第3274页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg8rt200600000001/#contentList
龙华新区
290
收集第3275页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg2rt200600000001/#contentList
龙华新区
290
收集第3276页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg11rt200600000001/#contentList
龙华新区
290
收集第3277页完毕
开始收集 https://sz.zu.ke.com/zufang/longhuaxinqu/pg13rt200600000001/#contentList
龙华新区
290
收集第3278页完毕
开始收集 

民治
278
收集第3357页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg21rt200600000001/#contentList
民治
278
收集第3358页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg10rt200600000001/#contentList
民治
278
收集第3359页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg9rt200600000001/#contentList
民治
279
收集第3360页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg6rt200600000001/#contentList
民治
286
收集第3361页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg11rt200600000001/#contentList
民治
286
收集第3362页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg9rt200600000001/#contentList
民治
286
收集第3363页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg18rt200600000001/#contentList
民治
286
收集第3364页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg3rt200600000001/#contentList
民治
288
收集第3365页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg22rt200600000001/#contentList
民治
288
收集第3366页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg6rt200600000001/#contentList
民治
293
收集第3367页完毕
开始收集 https://sz.zu.ke.com/zufang/minzhi/pg23rt200600000001/#contentList
民治
293
收集第3368页

梅林关
327
收集第3445页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg10rt200600000001/#contentList
梅林关
327
收集第3446页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg11rt200600000001/#contentList
梅林关
327
收集第3447页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg4rt200600000001/#contentList
梅林关
327
收集第3448页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg5rt200600000001/#contentList
梅林关
327
收集第3449页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg12rt200600000001/#contentList
梅林关
327
收集第3450页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg8rt200600000001/#contentList
梅林关
327
收集第3451页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg2rt200600000001/#contentList
梅林关
327
收集第3452页完毕
开始收集 https://sz.zu.ke.com/zufang/meilinguan/pg10rt200600000001/#contentList
梅林关
329
开始收集 https://sz.zu.ke.com/zufang/shangtang/pg1rt200600000001/#contentList
上塘
31
收集第3453页完毕
开始收集 https://sz.zu.ke.com/zufang/shangtang/pg2rt200600000001/#contentList
上塘
61
收集第3454页完毕
开始收集 https://sz.zu.ke.com/zufang/shangtang/pg3rt200600

石岩
19
收集第3535页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3536页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg1rt200600000001/#contentList
石岩
19
收集第3537页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3538页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg1rt200600000001/#contentList
石岩
19
收集第3539页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3540页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3541页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3542页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg1rt200600000001/#contentList
石岩
19
收集第3543页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg1rt200600000001/#contentList
石岩
19
收集第3544页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg1rt200600000001/#contentList
石岩
19
收集第3545页完毕
开始收集 https://sz.zu.ke.com/zufang/shiyan/pg2rt200600000001/#contentList
石岩
19
收集第3546页完毕
开始收集 https://sz

In [5]:
house_all_info[0][0]

['整租·万年青大厦',
 '1室1厅',
 '南/北',
 '罗湖区',
 '银湖',
 '万年青大厦',
 '34㎡',
 '南北',
 '1室1厅1卫',
 '中楼层（9层）',
 ' 官方核验 业主自荐 近地铁 随时看房 ',
 ' 3天前维护 ',
 '2600 元/月',
 '/zufang/SZ2741434419243401216.html']

In [6]:
final_house_info = []
for house_info in house_all_info:
    for house_single_info in house_info:
        final_house_info.append(house_single_info)

In [7]:
final_house_info[0]

['整租·万年青大厦',
 '1室1厅',
 '南/北',
 '罗湖区',
 '银湖',
 '万年青大厦',
 '34㎡',
 '南北',
 '1室1厅1卫',
 '中楼层（9层）',
 ' 官方核验 业主自荐 近地铁 随时看房 ',
 ' 3天前维护 ',
 '2600 元/月',
 '/zufang/SZ2741434419243401216.html']

In [16]:
df = pd.DataFrame(final_house_info)

In [17]:
df=df.drop_duplicates()

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,整租·万年青大厦,1室1厅,南/北,罗湖区,银湖,万年青大厦,34㎡,南北,1室1厅1卫,中楼层（9层）,官方核验 业主自荐 近地铁 随时看房,3天前维护,2600 元/月,/zufang/SZ2741434419243401216.html
1,整租·星湖花园,1室1厅,东,罗湖区,银湖,星湖花园,44㎡,东,1室1厅1卫,低楼层（28层）,公寓 近地铁 精装 随时看房,自如 1天前维护,4660 元/月,/zufang/SZ2447757759051988992.html
2,整租·星湖花园,1室1厅,南,罗湖区,银湖,星湖花园,37㎡,南,1室1厅1卫,中楼层（28层）,官方核验 新上 近地铁 随时看房,5天前维护,3500 元/月,/zufang/SZ2744897511595982848.html
3,整租·银晖名居,1室1厅,北,罗湖区,银湖,银晖名居,30㎡,北,1室1厅1卫,低楼层（19层）,公寓 近地铁 精装 随时看房,自如 1天前维护,3790 元/月,/zufang/SZ2694873795176972288.html
4,整租·华天公寓,2室1厅,东南,福田区,银湖,华天公寓,41㎡,东南,2室1厅1卫,中楼层（8层）,官方核验 业主自荐 近地铁 随时看房,5天前维护,3300 元/月,/zufang/SZ2724691184277790720.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76790,整租·鸿荣源·尚峻,2室2厅,复式,龙华区,上塘,鸿荣源·尚峻,65㎡,东东南南西南西,2室2厅2卫,高楼层（26层）,租住保障 公寓 月租 近地铁 精装 押一付一 双卫生间 随时看房,自如寓 1个月前维护,5199 元/月,/zufang/SZ2659263236662370304.html
76796,整租·鸿荣源·尚峻,3室2厅,复式,龙华区,上塘,鸿荣源·尚峻,94㎡,东东南南西南西,3室2厅2卫,中楼层（26层）,租住保障 公寓 月租 近地铁 精装 押一付一 双卫生间 随时看房,自如寓 1个月前维护,6399 元/月,/zufang/SZ2659254523214184448.html
76797,整租·鸿荣源·尚峻,2室2厅,复式,龙华区,上塘,鸿荣源·尚峻,93㎡,东东南南,2室2厅2卫,高楼层（26层）,租住保障 公寓 月租 近地铁 精装 押一付一 双卫生间 随时看房,龙华.筑梦自如寓 2天前维护,6149 元/月,/zufang/SZ2673735898973077504.html
76819,整租·汇邦名都花园,1室1厅,东南,宝安区,石岩,汇邦名都花园,42㎡,东南,1室1厅1卫,中楼层（31层）,官方核验 精装 随时看房,6天前维护,3500 元/月,/zufang/SZ2738374644100251648.html


In [18]:
print(final_house_info[0])

['整租·颐园', '5室2厅', '东', '罗湖区', '银湖', '颐园', '160㎡', '东', '5室2厅2卫', '低楼层（4层）', ' 官方核验 双卫生间 随时看房 ', ' 11天前维护 ', '23000 元/月', '/zufang/SZ2614228333625614336.html']


In [14]:
import pandas as pd 

In [15]:
df = pd.DataFrame(final_house_info)

In [16]:
pd.set_option('display.max_columns',25)
df=df.drop_duplicates()

In [14]:
len(df[df[4]=='0'])

0

In [19]:
df=df.drop_duplicates()
df.to_csv('贝壳0324_full.csv')